In [80]:
import numpy as np
%load_ext autoreload
%autoreload 2
gate = {
    "I": np.array([[1, 0], [0, 1]], dtype=np.complex128),
    "RX": lambda theta: np.array(
        [
            [np.cos(theta / 2), -1j * np.sin(theta / 2)],
            [-1j * np.sin(theta / 2), np.cos(theta / 2)],
        ],
        dtype=np.complex128,
    ),
    "RY": lambda theta: np.array(
        [
            [np.cos(theta / 2), -np.sin(theta / 2)],
            [np.sin(theta / 2), np.cos(theta / 2)],
        ],
        dtype=np.complex128,
    ),
    "RZ": lambda theta: np.array(
        [[np.exp(-1j * theta / 2), 0], [0, np.exp(1j * theta / 2)]], dtype=np.complex128
    ),
    "H": np.array([[1, 1], [1, -1]], dtype=np.complex128) / np.sqrt(2)
}

import qiskit

num_qubits = 3
qc = qiskit.QuantumCircuit(num_qubits)
qc.rz(0.3, 0)
qc.rz(0.4, 1)
qc.cx(0, 1)
qc.h(2)
qc.h(0)
qc.rx(0.5, 1)
qc.h(2)
qc.cx(0, 1)
qc.rz(0.5, 0)
qc.h(2)
from qimax import converter, circuit, splitter
num_qubits = qc.num_qubits
matrices = []
Us: list[circuit.U] = []
Usm = [] # [U_{0:m-1}, U_{1:m-1}, ... U_{m-1:m-1}]
index = 0


qasm_gates = converter.parse_qasm(converter.qc_to_qasm(qc))
Us = splitter.qasmgates_to_qcs2(qasm_gates)

gatess = [[0 for _ in range(num_qubits)] for _ in range(len(Us))]
for index, u in enumerate(Us):
    slots = np.ones(num_qubits)
    print('---')
    print(u)
    for gate in u:
        if gate[0] == 'CX':
            gatess[index][0] = ('CX', gate[1], gate[2])
        else:
            slots[gate[2]] = 0
            gatess[index][gate[2][0]] = (gate[0], gate[1], gate[2])
    for j, is_active in enumerate(slots):
        if is_active:
            gatess[index][j] = (['I', -999, [j]])
            
for u in gatess:
    print(u)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
---
[('RZ', 0.3, [0])]
---
[('RZ', 0.4, [1])]
---
[('CX', -999, [0, 1]), ('H', -999, [2])]
---
[('H', -999, [0]), ('RX', 0.5, [1]), ('H', -999, [2])]
---
[('CX', -999, [0, 1])]
---
[('RZ', 0.5, [0]), ('H', -999, [2])]
[('RZ', 0.3, [0]), ['I', -999, [1]], ['I', -999, [2]]]
[['I', -999, [0]], ('RZ', 0.4, [1]), ['I', -999, [2]]]
[['I', -999, [0]], ['I', -999, [1]], ('H', -999, [2])]
[('H', -999, [0]), ('RX', 0.5, [1]), ('H', -999, [2])]
[['I', -999, [0]], ['I', -999, [1]], ['I', -999, [2]]]
[('RZ', 0.5, [0]), ['I', -999, [1]], ('H', -999, [2])]


In [67]:
qasm_gates

[('RZ', 0.3, [0]),
 ('CX', -999, [0, 1]),
 ('H', -999, [2]),
 ('H', -999, [0]),
 ('RX', 0.5, [1]),
 ('H', -999, [2]),
 ('CX', -999, [0, 1]),
 ('RZ', 0.5, [0]),
 ('H', -999, [2])]

In [62]:
strs = []
strs.append(str(len(qcs)))
for qc in qcs:
    # Consider a U include n gates
    for gate1 in qc:
        print(gate1[0])
        if gate1[0] in ['I', 'RZ']:
            # Sparse
            if gate1[0] in ['I']:
                gate_data = [0, gate1[2][0], (gate[gate1[0]][0][0]), (gate[gate1[0]][1][1])]
            if gate1[0] in ['RZ']:
                param = gate1[1]
                gate_data = [0, gate1[2][0], gate[gate1[0]](param)[0][0], gate[gate1[0]](param)[1][1]]
        if gate1[0] in ['RX', 'RY', 'H']:
            # Dense
            if gate1[0] in ['RX', 'RY']:
                param = gate1[1]
                gate_data = [1, gate1[2][0], gate[gate1[0]](param)[0][0] + gate[gate1[0]](param)[0][1],
                         gate[gate1[0]](param)[1][0] + gate[gate1[0]](param)[1][1]]
            if gate1[0] in ['H']:
                gate_data = [1, gate1[2][0], gate[gate1[0]][0][0] + gate[gate1[0]][0][1],
                         gate[gate1[0]][1][0] + gate[gate1[0]][1][1]]
        if gate1[0] in ['X']:
            gate_data = [2, str(gate1[2][0]), (gate1[2][1])]
        for i in range(len(gate_data)):
            strs.append(gate_data[i])


with open('output.txt', 'w') as f:
    for item in strs:
        f.write("%s\n" % item)

RZ


TypeError: tuple indices must be integers or slices, not str

In [82]:
from qiskit import QuantumCircuit, transpile
from qiskit.converters import circuit_to_dag

def qc_to_qcs(qc):
	def split_layer_into_subcircuits(layer_ops, num_qubits):
		"""
		Given a list of DAG operations (from one layer), split them into 
		depth-1 QuantumCircuits such that each circuit satisfies:
		- It contains at most one one-qubit rotation gate (rx, ry, or rz)
			possibly combined with other one-qubit (non-parameterized) gates,
			OR
		- It contains a single two-qubit gate (e.g. a cx gate).
		
		Parameters:
		layer_ops: list of DAGOpNode objects from one layer.
		num_qubits: number of qubits in the original circuit.
		
		Returns:
		A list of QuantumCircuit objects (each depth-1 and meeting the constraints).
		"""
		subcircuits = []
		
		# ----- 1. Handle multi-qubit gates (e.g. cx) -----
		# Each two-qubit gate (or in general, any gate acting on >1 qubit)
		# must be isolated.
		for op in layer_ops:
			if len(op.qargs) > 1:
				qc_temp = QuantumCircuit(num_qubits)
				qc_temp.append(op.op, op.qargs, op.cargs)
				subcircuits.append(qc_temp)
		
		# ----- 2. Handle one-qubit gates -----
		one_qubit_ops = [op for op in layer_ops if len(op.qargs) == 1]
		rotation_names = ['rx', 'ry', 'rz']
		
		# Partition one-qubit ops into rotation and non-rotation gates.
		rotation_ops = []
		non_rotation_ops = []
		for op in one_qubit_ops:
			if op.op.name in rotation_names:
				rotation_ops.append(op)
			else:
				non_rotation_ops.append(op)
		
		# We want to group one-qubit ops so that each resulting circuit 
		# has at most one rotation.
		# Here’s one simple strategy:
		#   (a) For each rotation gate, create a circuit that includes that rotation.
		#       (Optionally, add any non-rotation gates that act on different qubits.)
		#   (b) Then, combine any leftover non-rotation gates into a single circuit.
		
		# (a) Process each rotation op:
		used_non_rotation = set()  # to mark non-rotation ops we assign along with a rotation
		
		for rot in rotation_ops:
			qc_temp = QuantumCircuit(num_qubits)
			qc_temp.append(rot.op, rot.qargs, rot.cargs)
			# Add any non-rotation op that is on a different qubit than the rotation.
			# (In a DAG layer, these should be disjoint—but we check just in case.)
			for i, op in enumerate(non_rotation_ops):
				if i not in used_non_rotation:
					if op.qargs[0] != rot.qargs[0]:
						qc_temp.append(op.op, op.qargs, op.cargs)
						used_non_rotation.add(i)
			subcircuits.append(qc_temp)
		
		# (b) For any remaining non-rotation ops not merged above, 
		# group them together in one circuit.
		remaining_non_rotation = [op for i, op in enumerate(non_rotation_ops) if i not in used_non_rotation]
		if remaining_non_rotation:
			qc_temp = QuantumCircuit(num_qubits)
			for op in remaining_non_rotation:
				qc_temp.append(op.op, op.qargs, op.cargs)
			subcircuits.append(qc_temp)
		
		return subcircuits


	# Convert the circuit to a DAG and extract its layers.
	dag = circuit_to_dag(qc)
	layers = list(dag.layers())

	qcs = []
	for layer in layers:
		# layer['graph'].op_nodes() returns the operations in that layer.
		layer_ops = list(layer['graph'].op_nodes())
		subcirs = split_layer_into_subcircuits(layer_ops, qc.num_qubits)
		qcs.extend(subcirs)
	return qcs

qcs = qc_to_qcs(qc)
for idx, circ in enumerate(qcs):
    print(f"\n--- Subcircuit {idx+1} ---")
    print(circ.draw())



--- Subcircuit 1 ---
     ┌─────────┐
q_0: ┤ Rz(0.3) ├
     └─────────┘
q_1: ───────────
        ┌───┐   
q_2: ───┤ H ├───
        └───┘   

--- Subcircuit 2 ---
                
q_0: ───────────
     ┌─────────┐
q_1: ┤ Rz(0.4) ├
     └─────────┘
q_2: ───────────
                

--- Subcircuit 3 ---
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘
q_2: ─────
          

--- Subcircuit 4 ---
          
q_0: ─────
          
q_1: ─────
     ┌───┐
q_2: ┤ H ├
     └───┘

--- Subcircuit 5 ---
        ┌───┐   
q_0: ───┤ H ├───
     ┌──┴───┴──┐
q_1: ┤ Rx(0.5) ├
     └──┬───┬──┘
q_2: ───┤ H ├───
        └───┘   

--- Subcircuit 6 ---
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘
q_2: ─────
          

--- Subcircuit 7 ---
     ┌─────────┐
q_0: ┤ Rz(0.5) ├
     └─────────┘
q_1: ───────────
                
q_2: ───────────
                


In [91]:
for gate in qcs[0].data:
    print(gate.name)
    print(gate.params)
    print(gate.qubits)

rz
[0.3]
(Qubit(QuantumRegister(3, 'q'), 0),)
h
[]
(Qubit(QuantumRegister(3, 'q'), 2),)


In [ ]:
def extract_circuit_info(circuit):
    circuit_list = []
    for instr, qargs, _ in circuit.data:
        gate = instr.name
        parameters = instr.params  # list of parameters (if any)
        print(qargs.)
        wires = [qubit for qubit in qargs]
        circuit_list.append((gate, parameters, wires))
    return circuit_list

# Extract the circuit information
circuit_info = extract_circuit_info(qcs[0])
for info in circuit_info:
	print(info)

C:\Users\haime\AppData\Local\Temp\ipykernel_2336\3434704346.py:3: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for instr, qargs, _ in circuit.data:


AttributeError: 'list' object has no attribute 'indices'